In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score


In [ ]:
data_set_base = pd.read_parquet("../02_preparación de los datos/data_set_final.parquet")

print(data_set_base.shape)

# Separacion de los datos

In [ ]:
X = data_set_base.drop(columns=["TARGET", "SK_ID_CURR"])
y = data_set_base["TARGET"]


# División del dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape)
print("Test:", X_test.shape)
print("Proporción default train:", y_train.mean())
print("Proporción default test:", y_test.mean())

# Modelado con regresión logistica

In [ ]:
log_reg = LogisticRegression(max_iter=1000, class_weight="balanced", n_jobs=-1)

log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)
y_proba = log_reg.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))


## Resultados del Modelo Base (Regresión Logística)

Con el objetivo de validar la calidad del dataset preparado y verificar la existencia de señal predictiva, se entrenó un modelo base de **Regresión Logística** utilizando el conjunto de datos final (`data_set_base`).  
Debido al fuerte desbalance de la variable objetivo (`TARGET` ≈ 8% de incumplimiento), se utilizó el parámetro `class_weight="balanced"` para priorizar la detección de la clase minoritaria.

### Reporte de clasificación

- **Clase 0 (clientes sin incumplimiento):**
  - Alta precisión (0.94)
  - Recall moderado (0.65)

- **Clase 1 (clientes con incumplimiento):**
  - Recall elevado (0.54), lo que indica que el modelo logra identificar más del 50% de los clientes en riesgo.
  - Precisión baja (0.12), esperable en un contexto altamente desbalanceado.

### Análisis e interpretación

El valor de **ROC AUC = 0.63** confirma que el modelo presenta una capacidad de discriminación superior al azar, lo cual es especialmente relevante considerando que se trata de un modelo lineal simple y sin optimización de hiperparámetros.  
Este resultado valida que el proceso de preparación de datos fue exitoso y permitió capturar patrones significativos asociados al incumplimiento de pago.

Desde una perspectiva de negocio, el **recall de la clase positiva** adquiere mayor relevancia que la precisión, dado que el costo de aprobar a un cliente riesgoso es superior al de rechazar a un cliente solvente. En este sentido, el desempeño del modelo es consistente con los objetivos del análisis de riesgo crediticio.

### Conclusión

Los resultados obtenidos permiten concluir que:

- El dataset preparado contiene **señal predictiva real**.
- Las transformaciones, agregaciones y variables derivadas construidas a partir de las tablas secundarias aportan valor al modelo.
- El conjunto de datos se encuentra en condiciones adecuadas para continuar con modelos más complejos y análisis comparativos.

En consecuencia, se considera que la **fase inicial de preparación de datos ha sido completada exitosamente**, habilitando el paso a la etapa de modelado avanzado.


# Modelado con Random Forest